In [1]:
#Import all the necessary libraries
import cv2
import numpy as np
import time
import os
from pathlib import Path
import tensorflow as tf
from tensorflow import keras

In [2]:
#Import Directory for OpenCV screenshots
screenshot_dir = 'CV_Screenshots'
os.makedirs(screenshot_dir, exist_ok=True)

In [3]:
#PHASE 1: CAMERA ACCESS AND SETUP

def find_working_camera():
    print("🔍 Searching for working camera...\n")
    
    # Try with AVFoundation (Mac-specific)
    cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
        
    if cap.isOpened():
            # Try to read a frame
        ret, frame = cap.read()
        if ret and frame is not None:
            h, w = frame.shape[:2]
            print(f"✅ WORKS! ({w}x{h})")
            cap.release()
            return 1
        else:
            print("❌ Opens but can't grab frames")
    else:
        print("❌ Can't open")
        
    cap.release()
    
    return None

def test_camera_access():
    # Find working camera
    camera_index = find_working_camera()
    
    if camera_index is None:
        print("\n❌ ERROR: No working camera found!")
        print("💡 Troubleshooting:")
        print("   1. Close any apps using camera (Zoom, Teams, FaceTime)")
        print("   2. System Settings > Privacy & Security > Camera")
        print("      → Enable for Terminal/Python/VS Code")
        print("   3. Restart your Python kernel/terminal")
        return
    
    print(f"\n✅ Using camera {camera_index}")
    
    # Open camera with AVFoundation backend (better for Mac)
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    
    # Force better settings
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("✅ Camera opened successfully!")
    print("\n📋 Instructions:")
    print("   - Press 'q' to quit")
    print("   - Press 's' to save screenshot")
    print("   - ESC also quits")
    print("\nCamera window should open now...\n")
    
    # Get camera properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"📊 Camera Info:")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps}")
    
    screenshot_count = 0
    frame_count = 0
    start_time = time.time()
    
    # Main camera loop
    while True:
        # Read frame from camera
        ret, frame = cap.read()
        
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        frame_count += 1
        
        # Calculate actual FPS
        elapsed_time = time.time() - start_time
        if elapsed_time > 0:
            actual_fps = frame_count / elapsed_time
        else:
            actual_fps = 0
        
        # Add FPS counter to frame
        cv2.putText(
            frame, 
            f"FPS: {actual_fps:.1f}", 
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.7, 
            (0, 255, 0), 
            2
        )
        
        # Add instructions
        cv2.putText(
            frame, 
            "Press 'q' to quit, 's' for screenshot", 
            (10, height - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 
            0.5, 
            (255, 255, 255), 
            1
        )
        
        # Display the frame
        cv2.imshow('Friend Classifier - Phase 1', frame)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q') or key == 27:  # 'q' or ESC
            print("\n👋 Quitting...")
            break
        elif key == ord('s'):  # Save screenshot
            screenshot_count += 1
            filename = os.path.join(screenshot_dir, f'screenshot_{screenshot_count}.jpg')
            cv2.imwrite(filename, frame)
            print(f"📸 Screenshot saved: {filename}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()


In [4]:
# PHASE 2: FACE DETECTION

def load_face_detector():    
    cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    face_cascade = cv2.CascadeClassifier(cascade_path)
    
    if face_cascade.empty():
        print("❌ ERROR: Could not load face detector!")
        print(f"   Tried path: {cascade_path}")
        return None
    
    print("✅ Face detector loaded successfully!")
    return face_cascade


def detect_faces(frame, face_cascade, scale_factor=1.1, min_neighbors=5, min_size=(30, 30)):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=scale_factor,
        minNeighbors=min_neighbors,
        minSize=min_size,
        flags=cv2.CASCADE_SCALE_IMAGE
    )
    return faces

In [5]:
# PHASE 3: FACE PREPROCESSING

def create_output_folder(folder_name="CV_PrePrep_Faces"):
    folder_path = Path(folder_name)
    folder_path.mkdir(exist_ok=True)
    
    print(f"📁 Output folder ready: {folder_path.absolute()}")
    
    return folder_path


def crop_face(frame, face_coords, margin=20): 
    x, y, w, h = face_coords
    frame_height, frame_width = frame.shape[:2]
    
    # Add margin but stay within frame bounds
    x1 = max(0, x - margin)
    y1 = max(0, y - margin)
    x2 = min(frame_width, x + w + margin)
    y2 = min(frame_height, y + h + margin)
    
    # Crop the face
    face_crop = frame[y1:y2, x1:x2]
    
    actual_coords = (x1, y1, x2-x1, y2-y1)
    
    return face_crop, actual_coords


def preprocess_face_for_model(face_crop, target_size=(128, 128), normalize=True):
    # Step 1: Convert BGR to RGB (OpenCV uses BGR, model expects RGB)
    face_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    
    # Step 2: Resize to model's expected input size
    face_resized = cv2.resize(face_rgb, target_size, interpolation=cv2.INTER_AREA)
    
    # Step 3: Create display version (before normalization)
    display_face = face_resized.copy()
    
    # Step 4: Normalize pixel values to 0-1 range (if needed)
    if normalize:
        face_normalized = face_resized.astype(np.float32) / 255.0
    else:
        face_normalized = face_resized.astype(np.float32)
    
    # Step 5: Add batch dimension (model expects: batch_size, height, width, channels)
    face_batched = np.expand_dims(face_normalized, axis=0)
    
    return face_batched, display_face


def save_preprocessed_face(face_image, folder_path, prefix="face", timestamp=None):
    if timestamp is None:
        timestamp = time.strftime("%Y%m%d_%H%M%S")
    
    # Generate unique filename
    filename = folder_path / f"{prefix}_{timestamp}.jpg"
    
    # Convert RGB to BGR for saving (OpenCV saves in BGR)
    if face_image.dtype == np.float32:
        # If normalized, denormalize first
        face_image = (face_image * 255).astype(np.uint8)
    
    face_bgr = cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR)
    
    # Save
    cv2.imwrite(str(filename), face_bgr)
    
    return filename


def visualize_preprocessing_pipeline(face_crop, target_size=(128, 128)):
    # Original (BGR)
    step1 = face_crop
    
    # Convert to RGB
    step2 = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    
    # Resize
    step3 = cv2.resize(step2, target_size)
    
    # Normalize (for visualization, convert back to 0-255)
    step4_normalized = step3.astype(np.float32) / 255.0
    step4 = (step4_normalized * 255).astype(np.uint8)
    
    # Create visualization
    h, w = 200, 200
    
    # Resize all to same size for display
    display1 = cv2.resize(step1, (w, h))
    display2 = cv2.resize(cv2.cvtColor(step2, cv2.COLOR_RGB2BGR), (w, h))
    display3 = cv2.resize(cv2.cvtColor(step3, cv2.COLOR_RGB2BGR), (w, h))
    display4 = cv2.resize(cv2.cvtColor(step4, cv2.COLOR_RGB2BGR), (w, h))
    
    # Add labels
    def add_label(img, text):
        labeled = img.copy()
        cv2.putText(labeled, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                   0.7, (0, 255, 0), 2)
        return labeled
    
    display1 = add_label(display1, "1. Original")
    display2 = add_label(display2, "2. BGR->RGB")
    display3 = add_label(display3, f"3. Resize {target_size}")
    display4 = add_label(display4, "4. Normalized")
    
    # Combine horizontally
    top_row = np.hstack([display1, display2])
    bottom_row = np.hstack([display3, display4])
    combined = np.vstack([top_row, bottom_row])
    
    return combined

In [6]:
# PHASE 4: MODEL LOADING AND PREDICTION

def load_trained_model(model_path="Models/friend_classifier_v2.keras"):

    try:
        print(f"🔄 Loading model from {model_path}...")
        model = keras.models.load_model(model_path)
        print("✅ Model loaded successfully!")
        return model
    
    except Exception as e:
        print(f"❌ ERROR loading model: {e}")
        print(f"   Make sure '{model_path}' exists in the current directory")
        return None


def predict_friend(model, face_processed, class_names, confidence_threshold=0.3):
    try:
        # Make prediction
        predictions = model.predict(face_processed, verbose=0)
        
        # Get predicted class and confidence
        predicted_class_idx = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class_idx]
        
        # Get predicted name
        predicted_name = class_names[predicted_class_idx]
        
        # Create dictionary of all predictions
        all_predictions = {
            class_names[i]: float(predictions[0][i]) 
            for i in range(len(class_names))
        }
        
        # If confidence is too low, return "Unknown"
        if confidence < confidence_threshold:
            predicted_name = "Unknown"
        
        return predicted_name, float(confidence), all_predictions
    
    except Exception as e:
        print(f"❌ Prediction error: {e}")
        return "Error", 0.0, {}


def draw_prediction_box(frame, face_coords, name, confidence, color=None):
    x, y, w, h = face_coords
    
    # Color coding based on confidence
    if color is None:
        if confidence >= 0.7:
            color = (0, 255, 0)  # Green - High confidence
        elif confidence >= 0.5:
            color = (0, 255, 255)  # Yellow - Medium confidence
        elif confidence >= 0.3:
            color = (0, 165, 255)  # Orange - Low confidence
        else:
            color = (0, 0, 255)  # Red - Very low confidence
    
    # Draw rectangle
    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 3)
    
    # Prepare label
    label = f"{name} ({confidence*100:.1f}%)"
    
    # Calculate label background size
    (label_w, label_h), baseline = cv2.getTextSize(
        label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2
    )
    
    # Draw label background
    cv2.rectangle(
        frame,
        (x, y - label_h - 15),
        (x + label_w + 10, y),
        color,
        -1  # Filled
    )
    
    # Draw label text
    cv2.putText(
        frame,
        label,
        (x + 5, y - 8),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,
        (255, 255, 255),  # White text
        2
    )


def display_top_predictions(frame, all_predictions, x_offset=10, y_offset=120):
    # Sort predictions by confidence
    sorted_preds = sorted(all_predictions.items(), key=lambda x: x[1], reverse=True)
    
    # Draw title
    cv2.putText(
        frame,
        "Top Predictions:",
        (x_offset, y_offset),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.6,
        (255, 255, 255),
        2
    )
    
    # Draw top 3
    for i, (name, conf) in enumerate(sorted_preds[:3]):
        y_pos = y_offset + 30 + (i * 25)
        text = f"{i+1}. {name}: {conf*100:.1f}%"
        
        # Color based on rank
        if i == 0:
            color = (0, 255, 0)  # Green
        elif i == 1:
            color = (0, 255, 255)  # Yellow
        else:
            color = (255, 255, 255)  # White
        
        cv2.putText(
            frame,
            text,
            (x_offset, y_pos),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            color,
            1
        )

In [7]:

# PHASE 5: FULL PIPELINE INTEGRATION

def friend_detector_with_model(camera_index=1, model_path="friend_classifier.keras"):
    # Class names for your 8 friends
    CLASS_NAMES = ["Alexa", "Emman", "Enzo", "Joshua", "Migy", "Rafiq", "Vaun", "Zoe"]
    
    # Load components
    print("=" * 60)
    print("🚀 FRIEND DETECTOR - Phase 4: Model Integration")
    print("=" * 60)
    
    # Load face detector
    face_cascade = load_face_detector()
    if face_cascade is None:
        return
    
    # Load model
    model = load_trained_model(model_path)
    if model is None:
        return
    
    # Setup output folder
    output_folder = create_output_folder("CV_Predictions")
    
    # Open camera
    cap = cv2.VideoCapture(camera_index, cv2.CAP_AVFOUNDATION)
    if not cap.isOpened():
        print("❌ Failed to open camera")
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    print("\n📋 Controls:")
    print("   q - Quit")
    print("   s - Save prediction")
    print("   t - Toggle top predictions display")
    print("   c - Cycle confidence threshold")
    print("\nWindow opening...\n")
    
    # State variables
    show_top_predictions = True
    confidence_thresholds = [0.3, 0.5, 0.7]
    current_threshold_idx = 0
    confidence_threshold = confidence_thresholds[current_threshold_idx]
    
    saved_count = 0
    frame_count = 0
    start_time = time.time()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Detect faces
        faces = detect_faces(frame, face_cascade)
        
        # Process each detected face
        for i, (x, y, w, h) in enumerate(faces):
            # Crop and preprocess face
            face_crop, actual_coords = crop_face(frame, (x, y, w, h), margin=20)
            
            if face_crop.size == 0:
                continue
            
            # Preprocess for model
            face_processed, face_display = preprocess_face_for_model(
                face_crop,
                target_size=(128, 128),
                normalize=True
            )
            
            # Make prediction
            predicted_name, confidence, all_predictions = predict_friend(
                model,
                face_processed,
                CLASS_NAMES,
                confidence_threshold=confidence_threshold
            )
            
            # Draw prediction box
            draw_prediction_box(frame, (x, y, w, h), predicted_name, confidence)
            
            # Display preprocessed face in corner (only for first face)
            if i == 0:
                corner_size = 150
                face_corner = cv2.resize(
                    cv2.cvtColor(face_display, cv2.COLOR_RGB2BGR),
                    (corner_size, corner_size)
                )
                
                # Position in top-right corner
                frame[10:10+corner_size, frame.shape[1]-corner_size-10:frame.shape[1]-10] = face_corner
                
                # Add border
                cv2.rectangle(
                    frame,
                    (frame.shape[1]-corner_size-10, 10),
                    (frame.shape[1]-10, 10+corner_size),
                    (255, 255, 0),
                    2
                )
                
                # Show top predictions if enabled
                if show_top_predictions:
                    display_top_predictions(frame, all_predictions)
        
        # Add overlay info
        fps = frame_count / (time.time() - start_time)
        
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.putText(frame, f"Faces: {len(faces)}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.putText(frame, f"Threshold: {confidence_threshold:.1f}", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Controls help
        cv2.putText(frame, "q: Quit | s: Save | t: Toggle | c: Threshold",
                   (10, frame.shape[0]-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Display
        cv2.imshow('Friend Detector - Phase 4: COMPLETE!', frame)
        
        # Handle input
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        
        elif key == ord('s') and len(faces) > 0:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = output_folder / f"prediction_{predicted_name}_{timestamp}.jpg"
            cv2.imwrite(str(filename), frame)
            saved_count += 1
            print(f"💾 Saved: {filename}")
        
        elif key == ord('t'):
            show_top_predictions = not show_top_predictions
            print(f"📊 Top predictions: {'ON' if show_top_predictions else 'OFF'}")
        
        elif key == ord('c'):
            current_threshold_idx = (current_threshold_idx + 1) % len(confidence_thresholds)
            confidence_threshold = confidence_thresholds[current_threshold_idx]
            print(f"🎯 Confidence threshold: {confidence_threshold:.1f}")
    
    # Cleanup
    cap.release()
    cv2.destroyAllWindows()
    
    print(f"\n✅ Session complete! Saved {saved_count} predictions")

In [8]:
if __name__ == "__main__":
    friend_detector_with_model(
        camera_index=1,
        model_path="Models/friend_classifier_v2.keras"  # Make sure this file is in your directory!
    )

🚀 FRIEND DETECTOR - Phase 4: Model Integration
✅ Face detector loaded successfully!
🔄 Loading model from Models/friend_classifier_v2.keras...
✅ Model loaded successfully!
📁 Output folder ready: /Users/enzobasuil/Desktop/Coding Projects/Python Projects/FriendIdentifier/CV_Predictions

📋 Controls:
   q - Quit
   s - Save prediction
   t - Toggle top predictions display
   c - Cycle confidence threshold

Window opening...


✅ Session complete! Saved 0 predictions
